In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import affine_transform
from sklearn.neighbors import KNeighborsClassifier

In [2]:

# Load MNIST dataset
def load_mnist():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0,1]
    return x_train, y_train, x_test, y_test

# Apply non-linear deformation and shiftable edges
def apply_deformation(images):
    transformed_images = []
    for img in images:
        # Generate a small random affine transformation matrix
        matrix = np.array([[1 + np.random.uniform(-0.2, 0.2), np.random.uniform(-0.2, 0.2), 0],
                           [np.random.uniform(-0.2, 0.2), 1 + np.random.uniform(-0.2, 0.2), 0],
                           [0, 0, 1]])
        transformed_img = affine_transform(img, matrix[:2, :2], offset=matrix[:2, 2], mode='constant', cval=0.0)
        transformed_images.append(transformed_img)
    return np.array(transformed_images)

# Preprocess data
def preprocess_data():
    x_train, y_train, x_test, y_test = load_mnist()
    x_train = apply_deformation(x_train)
    x_test = apply_deformation(x_test)
    
    # Flatten images
    x_train = x_train.reshape(len(x_train), -1)
    x_test = x_test.reshape(len(x_test), -1)
    
    # Standardize the dataset
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    return x_train, y_train, x_test, y_test

# Train KNN classifier
def train_knn(x_train, y_train, k=3):
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn.fit(x_train, y_train)
    return knn

# Evaluate the model
def evaluate_model(knn, x_test, y_test):
    accuracy = knn.score(x_test, y_test)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [3]:

if __name__ == "__main__":
    x_train, y_train, x_test, y_test = preprocess_data()
    knn_model = train_knn(x_train, y_train, k=3)
    evaluate_model(knn_model, x_test, y_test)


c:\Users\noams\anaconda3\envs\knn_env\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\noams\anaconda3\envs\knn_env\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\noams\anaconda3\envs\knn_env\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\noams\anaconda3\envs\knn_env\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\noams\anaconda3\envs\knn_env\lib\subprocess.py", line 1456, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess

Test Accuracy: 0.8289
